<a href="https://colab.research.google.com/github/petikmaryna/DSL/blob/main/Job3Petik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(за основу були взяті матеріали з [цієї](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9F%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5_FIRST_%D0%B8_FOLLOW) та [цієї](https://www.cs.uaf.edu/~cs331/notes/FirstFollow.pdf) сторінок)

In [1]:

grammar = {
    'toks': set( [
        ('bracket', '('), 
        ('bracket', ')'), 
        ('plus', '+'), 
        ('mul', '*'),
        ('0', '')
    ] ),
    'vars': {
        'E' : [['T', 'E1']],
        'E1' : [[('plus', '+'), 'T', 'E1'],
               [('0', '')]],
        'T' : [['F', 'T1']],
        'T1' : [[('mul', '*'), 'F', 'T1'],
               [('0', '')]],
        'F' : [[('bracket', '('), 'E', ('bracket', ')')],
               [('0', '')]]
    },
    'hvar': 'E'
}

# **1. Програма, що генерує функцію FIRST(Х)**
Якщо А — це рядок із символів, що входять до Граматики, тоді FIRST(А) — це множина терміналів, з яких починаються рядки, що походять з А. Якщо A ⇒ ε, тоді ε також належить FIRST(А).
Щоб обчислити FIRST(X) для всіх X символів, що входять до Граматики, треба застосувати наступні правила, доки не залишилося терміналів або ε, які можна додати до будь-якої множини FIRST(X): 
1. Якщо X термінал, тоді FIRST(X) це {X}.
2. Якщо X ⇒ ε правило, тоді треба додати ε до FIRST(X).
3. Якщо X нетермінал і X ⇒ Y1, Y2 ... Yk правило, тоді треба додати а до FIRST(X), якщо для якого-небудь і а входить до FIRST(Yi), а також ε належить до кожної з множин FIRST(Y1), ... , FIRST(Yi-1), що результує в Y1, ... ,Yi-1 ⇒ ε. 

Якщо ε належить до FIRST(Yj) для кожного j = 1, 2, ... , k, тоді треба додати ε до FIRST(X)(тобто усі елементи множини FIRST(Y1) також належать до FIRST(X)). Якщо Y1 не переходить в ε, тоді до FIRST(X) нічого більше не додається, але якщо Y1 ⇒ ε, тоді ми додаємо FIRST(Y2) і так далі.

In [ ]:
def first(X, rules, tokens):
  resultingSet = set()
  
  if len(rules) == 0:
    return resultingSet 
  
  for rule in rules[X]:
    if rule[0] in tokens:
      resultingSet.add(rule[0])
    else: 
      resultingSet = resultingSet.union(first(rule[0], rules, tokens)) 
      
  return resultingSet

# **2. Програма, що генерує функцію FOLLOW**

Для нетерміналу A визначимо FOLLOW(A) як множину терміналів, які можуть з`явитися відразу справа до А у якій-небуть сентенційній конструкції, тобто таку множину темніналів, для яких існує правило форми  S ⇒ aAab для деяких a і b. Якщо А може з`явитися з правого краю в якій-небудь сентенційній формі, тоді до FOLLOW(A) додається пустий символ, репрезентуючий маркер правого кінця.
Щоб знайти FOLLOW(A) треба слідувати алгоритму до тих пір, доки нічого нового не додасться до множини:
1. Треба додати (пустий символ) до FOLLOW(s), де S стартовий символ і (пустий символ) маркер лівого кінця.
2. Якщо існує правило A ⇒ aBb, тоді усі елементи множини FIRST(b), окрім e, додається до FOLLOW(B).
3. Якщо існує правило A ⇒ aB, або правило A ⇒ aBb, де FIRST(b) включає e (i.e., b ⇒ e), тоді усі елементи множини FOLLOW(A) належать і множині FOLLOW(B).

In [ ]:
def FOLLOWALL(grammar):

  def FIRSTforSeq(seq):
    result = set()
    for symbol in seq:
      if symbol in tokens:  
        result.add(symbol)
        return result
      else:
        result = set.union(result, dictFIRST[symbol])
        if endlSymb not in dictFIRST[symbol]:
          return result
        else:
          result.remove(endlSymb) 

    result.add(endlSymb)  
    return result
 

  nonterms = grammar['vars'].keys()
  tokens = grammar['toks']
  startNonTerm = grammar['hvar']

  resultDict = {}
  for nt in nonterms:
    resultDict[nt] = (set())

  dictFIRST = {}
  for nt in nonterms:
    dictFIRST[nt] = first(nt, grammar['vars'], tokens)
  
  endlSymb = ('0', '')
  resultDict[startNonTerm].add(endlSymb)

  changed = True
  while changed:
    changed = False
    for nonterm, rules in grammar['vars'].items():
      for rule in rules:
        for index, symbol in enumerate(rule):
          if symbol in nonterms:
            endSeqFIRST =  FIRSTforSeq(rule[index+1:])
            prevLen = len(resultDict[symbol])

            if endlSymb in endSeqFIRST:
              endSeqFIRST.remove(endlSymb)
              resultDict[symbol] = resultDict[symbol].union(resultDict[nonterm])

            resultDict[symbol] = resultDict[symbol].union(endSeqFIRST) 
                                                                      

            newLen = len(resultDict[symbol])
            if newLen != prevLen:
              changed = True
  return resultDict

In [ ]:
print(first('E1', grammar['vars'], grammar['toks']))
print(FOLLOWALL(grammar))

In [ ]:
{('plus', '+'), ('0', '')}
{'E': {('bracket', ')'), ('0', '')},
 'E1': {('bracket', ')'), ('0', '')}, 
 'T': {('bracket', ')'), ('plus', '+'), ('0', '')}, 
 'T1': {('bracket', ')'), ('plus', '+'), ('0', '')}, 
 'F': {('bracket', ')'), ('mul', '*'), ('plus', '+'), ('0', '')}}